In [ ]:
import os
import re
from collections import defaultdict
from nltk.stem import PorterStemmer
import json

inverted_index = defaultdict(list)
stemmer = PorterStemmer()
def preprocess_text(text):
   text = text.lower()
   text = re.sub('<[^<]+?>', '', text)
   tokens = re.findall(r'\b\w+\b', text)
   tokens = [stemmer.stem(token) for token in tokens]
   return tokens


def build_inverted_index(directory):
    file_num =0 
    tag_weights = {'title': 3, 'h1': 2.5, 'h2': 2, 'h3': 1.5, 'b': 1, 'strong': 1} #tags' weight
    for subdir, files in os.walk(directory):
        for file in files:
            path = subdir.replace("\\", "//")
            if file.endswith('.json'):
                try:
                    with open(os.path.join(path, file).replace("\\", "//"), 'r', encoding="utf-8") as f:
                            data = json.load(f)
                    file_num += 1
                    doc_id = data['url']
                    doc_content = data['content']
                    #url = str(data['url'])
                    #url.replace("db"," ")
                    #url.replace("cs"," ")
                    #url = re.sub(r"https?://(?:www\.)?", "", url)
                    #url = url.rstrip("/")
                    #url = re.sub(r"[^\w\s]", " ", url)
                    #tokens = preprocess_text(doc_content)+url.split()
                    tokens = preprocess_text(doc_content)
                    tf = dict()
                    for token in tokens:
                        if token not in tf.keys():
                            tf[token]=1
                        else:
                            tf[token]+=1
                            
                    #iterate every word excpet tags
                    for tag, weight in tag_weights.items():
                        tag_regex = fr'<{tag}.*?>(.*?)<\/{tag}>'
                        matches = re.findall(tag_regex, doc_content, flags=re.IGNORECASE)
                        for match in matches:
                            match_tokens = preprocess_text(match)
                            for t in match_tokens:
                                tf[t] += weight
                    # add document to inverted index
                    for token, frequency in tf.items():
                        inverted_index[token].append([file, doc_id, frequency])
                   
                except:
                    continue
    #idf = {}
    #for token, posting in inverted_index.items():
        #idf[token] = math.log((file_num + 0.1) / (0.1+ len(posting)))+0.1
        #for i in range(len(posting)):
            #posting[i][2] = posting[i][2] * (idf[token])
    print(file_num)
    return inverted_index
def load_index(url):
    with open(url, 'r') as f:
        return json.load(f)
def main():
    while True:
        choice = input("Do you have the inverted index file already? (y/n/q)")
        if choice.lower() == "y":
            filename = input("Please enter the file address: ")
            try:
                with open(filename, "r") as f:
                    inverted_index = eval(f.read())
                    break
            except (FileNotFoundError, SyntaxError):
                print("Invalid file address. Please try again.")
        elif choice.lower() == "n":
            directory = input("Please enter the directory where the html files at and save the inverted index file: ")
            try:
                os.makedirs(directory, exist_ok=True)
                filename = os.path.join(directory, "inverted_index.txt")
                inverted_index = build_inverted_index(directory)
                print(len(set(inverted_index.keys())))
                with open(filename, "w") as f:
                    json.dump(inverted_index, f, indent=2)
                break
            except OSError:
                print("Invalid directory. Please try again.")
        elif choice.lower() == "q":
            exit()
if __name__ == '__main__':
    main()